# Web Scraping I - Data formatted as a table

## Brief

Web Scraping allows us to extract data available on web sites.
Technically, it consists of reading and parsing the html code and then extracting the elements that are valuable to us. Basically, we transform the html input into an appropriate data structure to be processed.
One of the easiest cases and the one that we will deal with in this practice is the data already formatted as tables.
As an example, we will use the 'List of accidents and disasters by death toll' from Wikipedia and more specifically the table related to explosions. Clarification: This list does not include bombings, aviation incidents, or mining disasters.
Link: https://en.wikipedia.org/wiki/List_of_accidents_and_disasters_by_death_toll

## Import libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn')
%matplotlib inline

## Import  and loading data

In [ ]:
tables = pd.read_html("https://en.wikipedia.org/wiki/List_of_accidents_and_disasters_by_death_toll")
explosions = tables[4]

## Data overview

In [ ]:
explosions.head(10)

In [ ]:
explosions.shape

In [ ]:
explosions.info()

## Data cleansing

###  Set columns names according to row 1 (if necessary)

In [ ]:
columns_list = []
for num in range(0,explosions.shape[1]):
    columns_list.append(explosions[num][0])
    
columns_list

In [ ]:
explosions.columns = columns_list
explosions.drop(0,inplace=True)
explosions.head()

### Scrub 'Deaths' column data

#### Values formatted as a range

Since there are values that are shown as intervals, we will define 2 new columns "Deaths_min" and "Deaths_max" where we will extract the minimum and maximum range of them.

In [ ]:
## Function to extract the minimum range:

def min_interval(value):
    
    if '–' in value:
        return value[:value.find('–')]
    elif ' or ' in value:
        return value[:value.find(' or ')]
    else:
        return value

min_interval('1,400–2,280')

In [ ]:
## Function to extract the maximum range:

def max_interval(value):
    
    if '–' in value:
        return value[value.find('–')+1:]
    elif ' or ' in value:
        return value[value.find(' or ')+1:]
    else:
        return value

max_interval('1,400–2,280')

In [ ]:
## New columns creation with the 2 functions:

explosions['Deaths_min'] = explosions['Deaths'].apply(min_interval)
explosions['Deaths_max'] = explosions['Deaths'].apply(max_interval)
explosions.sample(10)

#### Remove non-numeric characters:

In [ ]:
# Function to remove non-numeric characters

def erase_char(value):
    result = ""
    for char in value:
        if char.isnumeric():
            result += char
    return result
            
erase_char("327(estimate)")

In [ ]:
## New columns creation with the function:

explosions['Deaths_min_clean']=pd.to_numeric(explosions['Deaths_min'].apply(erase_char))
explosions['Deaths_max_clean']=pd.to_numeric(explosions['Deaths_max'].apply(erase_char))
explosions.sample(10)

## Data analysis

Our data are ready to work with them. For instance, let is calculate how many people died in accidental explosions per decade in the XX century.

### Create 'Century' and 'Decade' column from 'Date'

In [ ]:
explosions['Century'] = explosions['Date'].str[-4:-2]

In [ ]:
explosions['Decade'] = explosions['Date'].str[-2:-1]

In [ ]:
explosions.sample(5)

### Filter the 20th century and save the results into a new dataframe

In [ ]:
twenty_century = explosions[explosions['Century']=='19']
twenty_century.sample(5)


In [ ]:
twenty_century.shape


### Group by 'Decades' and add the number of deaths for each of them

In [ ]:
deaths_per_decade_XX = twenty_century.groupby(twenty_century['Decade'].str[0])[['Deaths_min_clean','Deaths_max_clean']].sum()
deaths_per_decade_XX

### Plot results

In [ ]:
deaths_per_decade_XX.plot()

# THANK YOUR FOR FOLLOWING

If you have any question, do not hesitate to contact me on Linkedin: https://www.linkedin.com/in/laurajmoreno/